In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Dropout, Dense,
                                     BatchNormalization, Flatten, GlobalAveragePooling2D)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from skimage.transform import resize

# Define your folder structure
data_dir = '/content/drive/MyDrive/training_dataset'
classes = ['cat', 'dog','bird','cow','lion','sheep','frog','chicken','donkey','monkey', 'others']

# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []

    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.exists(class_dir):
            for filename in os.listdir(class_dir):
                if filename.endswith('.wav'):
                    file_path = os.path.join(class_dir, filename)
                    audio_data, sample_rate = librosa.load(file_path, sr=22050)

                    # Convert to Mel spectrogram
                    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate, n_mels=128)
                    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)  # Convert to dB scale

                    # Resize for CNN input
                    mel_spectrogram = resize(mel_spectrogram, target_shape, mode='constant')
                    mel_spectrogram = np.expand_dims(mel_spectrogram, axis=-1)  # Add channel dimension

                    data.append(mel_spectrogram)
                    labels.append(i)

    return np.array(data), np.array(labels)

# Data Augmentation (Time Shift, Noise Injection, Pitch Shift)
def augment_audio(audio, sr):
    if np.random.rand() < 0.3:
        audio = np.roll(audio, shift=int(sr * 0.1))  # Time Shift
    if np.random.rand() < 0.3:
        audio += 0.005 * np.random.randn(len(audio))  # Add Gaussian Noise
    if np.random.rand() < 0.3:
        audio = librosa.effects.pitch_shift(audio, sr=sr, n_steps=np.random.uniform(-2, 2))  # Pitch Shift
    return audio

# Load and split data
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define CNN model
input_shape = X_train.shape[1:]

input_layer = Input(shape=input_shape)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.3)(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.4)(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.4)(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.5)(x)

x = GlobalAveragePooling2D()(x)  # Prevents overfitting by reducing parameter count

x = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)

output_layer = Dense(len(classes), activation='softmax')(x)

model = Model(input_layer, output_layer)

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.2514 - loss: 7.1606

37/37 ━━━━━━━━━━━━━━━━━━━━ 163s 4s/step - accuracy: 0.2530 - loss: 7.1426 - val_accuracy: 0.1497 - val_loss: 16.7711 - learning_rate: 0.0010
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.4981 - loss: 4.9816

37/37 ━━━━━━━━━━━━━━━━━━━━ 201s 4s/step - accuracy: 0.4988 - loss: 4.9739 - val_accuracy: 0.2177 - val_loss: 6.8810 - learning_rate: 0.0010
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6035 - loss: 3.8822

37/37 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.6037 - loss: 3.8758 - val_accuracy: 0.4558 - val_loss: 3.7956 - learning_rate: 0.0010
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6983 - loss: 2.9668

37/37 ━━━━━━━━━━━━━━━━━━━━ 209s 5s/step - accuracy: 0.6979 - loss: 2.9643 - val_accuracy: 0.5068 - val_loss: 3.0585 - learning_rate: 0.0010
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 195s 4s/step - accuracy: 0.7379 - loss: 2.3653 - val_accuracy: 0.3878 - val_loss: 2.9844 - learning_rate: 0.0010
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7685 - loss: 1.9489

...

37/37 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9057 - loss: 0.6626 - val_accuracy: 0.7211 - val_loss: 1.2031 - learning_rate: 0.0010
Epoch 13/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9058 - loss: 0.6293 - val_accuracy: 0.3741 - val_loss: 3.9376 - learning_rate: 0.0010
Epoch 14/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 165s 4s/step - accuracy: 0.9284 - loss: 0.5529 - val_accuracy: 0.5408 - val_loss: 1.8373 - learning_rate: 0.0010
Epoch 15/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 160s 4s/step - accuracy: 0.9388 - loss: 0.4611 - val_accuracy: 0.5578 - val_loss: 1.8211 - learning_rate: 0.0010
Epoch 16/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 203s 4s/step - accuracy: 0.9261 - loss: 0.4824 - val_accuracy: 0.2211 - val_loss: 4.6200 - learning_rate: 0.0010
Epoch 17/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9549 - loss: 0.3832
Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
37/37 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9549 - loss: 0.3831 - val_accuracy: 0

37/37 ━━━━━━━━━━━━━━━━━━━━ 200s 4s/step - accuracy: 0.9666 - loss: 0.3300 - val_accuracy: 0.9218 - val_loss: 0.4704 - learning_rate: 5.0000e-04
Epoch 19/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 203s 4s/step - accuracy: 0.9724 - loss: 0.3049 - val_accuracy: 0.8844 - val_loss: 0.5252 - learning_rate: 5.0000e-04
Epoch 20/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9876 - loss: 0.2341 - val_accuracy: 0.8912 - val_loss: 0.5533 - learning_rate: 5.0000e-04
Epoch 21/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 198s 4s/step - accuracy: 0.9841 - loss: 0.2328 - val_accuracy: 0.8810 - val_loss: 0.5098 - learning_rate: 5.0000e-04
Epoch 22/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9875 - loss: 0.2077

37/37 ━━━━━━━━━━━━━━━━━━━━ 159s 4s/step - accuracy: 0.9876 - loss: 0.2076 - val_accuracy: 0.9252 - val_loss: 0.4401 - learning_rate: 5.0000e-04
Epoch 23/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 204s 4s/step - accuracy: 0.9955 - loss: 0.1784 - val_accuracy: 0.8741 - val_loss: 0.6100 - learning_rate: 5.0000e-04
Epoch 24/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9937 - loss: 0.1688

37/37 ━━━━━━━━━━━━━━━━━━━━ 201s 4s/step - accuracy: 0.9937 - loss: 0.1689 - val_accuracy: 0.9286 - val_loss: 0.3959 - learning_rate: 5.0000e-04
Epoch 25/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 205s 4s/step - accuracy: 0.9861 - loss: 0.1761 - val_accuracy: 0.8571 - val_loss: 0.5744 - learning_rate: 5.0000e-04
Epoch 26/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 198s 4s/step - accuracy: 0.9892 - loss: 0.1742 - val_accuracy: 0.9048 - val_loss: 0.4734 - learning_rate: 5.0000e-04
Epoch 27/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 161s 4s/step - accuracy: 0.9954 - loss: 0.1544 - val_accuracy: 0.8639 - val_loss: 0.5601 - learning_rate: 5.0000e-04
Epoch 28/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9916 - loss: 0.1531

37/37 ━━━━━━━━━━━━━━━━━━━━ 160s 4s/step - accuracy: 0.9916 - loss: 0.1530 - val_accuracy: 0.9558 - val_loss: 0.2984 - learning_rate: 5.0000e-04
Epoch 29/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 201s 4s/step - accuracy: 0.9908 - loss: 0.1425 - val_accuracy: 0.9218 - val_loss: 0.3501 - learning_rate: 5.0000e-04
Epoch 30/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 202s 4s/step - accuracy: 0.9834 - loss: 0.1447 - val_accuracy: 0.8776 - val_loss: 0.5761 - learning_rate: 5.0000e-04
Epoch 31/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 206s 4s/step - accuracy: 0.9938 - loss: 0.1358 - val_accuracy: 0.4762 - val_loss: 2.4858 - learning_rate: 5.0000e-04
Epoch 32/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 198s 4s/step - accuracy: 0.9917 - loss: 0.1387 - val_accuracy: 0.7959 - val_loss: 0.7057 - learning_rate: 5.0000e-04
Epoch 33/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9910 - loss: 0.1384
Epoch 33: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
37/37 ━━━━━━━━━━━━━━━━━━━━ 203s 4s/step - accuracy: 0.9910 - loss: 0.13

In [16]:

def predict_with_others_class(model, X_test, threshold=0.5):
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)  # Get the index of the highest probability

    # If the max probability is less than the threshold, classify as "others" (last class)
    max_probs = np.max(predictions, axis=1)
    others_class = len(classes) - 1  # "others" is the last class

    for i in range(len(predicted_classes)):
        if max_probs[i] < threshold:
            predicted_classes[i] = others_class  # Assign "others" if probability is less than threshold

    return predicted_classes


In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.7752442955970764


In [25]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation loss
    patience=10,          # Number of epochs with no improvement before stopping
    restore_best_weights=True # Restore model to best epoch
)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 33s 845ms/step - accuracy: 0.3957 - loss: 8.0373 - val_accuracy: 0.4218 - val_loss: 1.9139
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 31s 830ms/step - accuracy: 0.4008 - loss: 2.2176 - val_accuracy: 0.4524 - val_loss: 1.8587
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 41s 829ms/step - accuracy: 0.5495 - loss: 1.5292 - val_accuracy: 0.5034 - val_loss: 1.7569
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 41s 834ms/step - accuracy: 0.5718 - loss: 1.4154 - val_accuracy: 0.5238 - val_loss: 1.5769
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 31s 829ms/step - accuracy: 0.6279 - loss: 1.2437 - val_accuracy: 0.5544 - val_loss: 1.4759
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 42s 844ms/step - accuracy: 0.6642 - loss: 1.0845 - val_accuracy: 0.5850 - val_loss: 1.4106
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 40s 835ms/step - accuracy: 0.6543 - loss: 1.1507 - val_accuracy: 0.6054 - val_loss: 1.3945
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 32s 862ms/step - accuracy: 0.6931 - loss: 1.0438 - val_accu

In [22]:
model.save('awid_model_others.h5')

In [34]:
model = load_model('best_model.h5')

# Define the target shape for input spectrograms
target_shape = (128, 128)

# Define your class labels
classes = ['cat', 'dog','bird','cow','lion','sheep','frog','chicken','donkey','monkey','others']

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=16000)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))

    # Make predictions
    predictions = model.predict(mel_spectrogram)

    # Get the class probabilities
    class_probabilities = predictions[0]

    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)

    return class_probabilities, predicted_class_index

# Test an audio file
test_audio_file = 'cow.wav'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
Class: cat, Probability: 0.0000
Class: dog, Probability: 0.0000
Class: bird, Probability: 0.0000
Class: cow, Probability: 0.0000
Class: lion, Probability: 0.0000
Class: sheep, Probability: 0.0000
Class: frog, Probability: 1.0000
Class: chicken, Probability: 0.0000
Class: donkey, Probability: 0.0000
Class: monkey, Probability: 0.0000
Class: others, Probability: 0.0000
The audio is classified as: frog
Accuracy: 1.0000


In [39]:
import numpy as np
import librosa
import os
from skimage.transform import resize
import tensorflow as tf

# Define classes
classes = ['cat', 'dog', 'bird', 'cow', 'lion', 'sheep', 'frog', 'chicken', 'donkey', 'monkey', 'others']

# Load trained model
model = tf.keras.models.load_model('best_model.h5')  # Update with correct path

def preprocess_audio(file_path, target_shape=(128, 128)):
    audio_data, sr = librosa.load(file_path, sr=22050)

    # Convert to Mel spectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sr, n_mels=128)
    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

    # Resize to match CNN input
    mel_spectrogram = resize(mel_spectrogram, target_shape, mode='constant')
    mel_spectrogram = np.expand_dims(mel_spectrogram, axis=-1)  # Keep only 1 channel

    return np.expand_dims(mel_spectrogram, axis=0)  # Add batch dimension


# Function to predict audio file class
def predict_audio(file_path):
    preprocessed_audio = preprocess_audio(file_path)
    prediction = model.predict(preprocessed_audio)

    predicted_class = classes[np.argmax(prediction)]
    confidence = np.max(prediction) * 100  # Convert to percentage

    print(f'Predicted Class: {predicted_class} ({confidence:.2f}%)')

# Test on sample files
test_audio_files = [

    "dog.wav",
    "bird.wav",
    "bird1.wav",
    "frog.wav",
    "rain.mp3",
    "wind.mp3",
    "cow.wav"
]

for file in test_audio_files:
    if os.path.exists(file):
        print(f"\nTesting: {file}")
        predict_audio(file)
    else:
        print(f"File not found: {file}")



Testing: dog.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Predicted Class: dog (90.03%)

Testing: bird.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted Class: bird (93.51%)

Testing: bird1.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Predicted Class: bird (96.84%)

Testing: frog.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted Class: frog (99.88%)

Testing: rain.mp3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Class: others (99.73%)

Testing: wind.mp3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Predicted Class: others (68.90%)

Testing: cow.wav
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted Class: cow (99.51%)


In [38]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Get model predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax probabilities to class labels
y_true_classes = np.argmax(y_test, axis=1)  # Convert one-hot encoding back to labels

# Print classification report
print(classification_report(y_true_classes, y_pred_classes, target_names=classes))


10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 770ms/step
              precision    recall  f1-score   support

         cat       0.94      0.88      0.91        34
         dog       0.97      0.86      0.92        44
        bird       0.98      1.00      0.99        45
         cow       1.00      1.00      1.00        19
        lion       0.89      1.00      0.94        17
       sheep       0.87      1.00      0.93        13
        frog       1.00      1.00      1.00        18
     chicken       0.88      1.00      0.94        22
      donkey       1.00      0.92      0.96        12
      monkey       0.95      0.90      0.92        20
      others       0.98      1.00      0.99        50

    accuracy                           0.96       294
   macro avg       0.95      0.96      0.95       294
weighted avg       0.96      0.96      0.96       294



In [33]:
import numpy as np
from sklearn.metrics import classification_report

# Convert one-hot encoded y_test to class indices (integer labels)
y_test_class = np.argmax(y_test, axis=1)

# Predict the class probabilities
y_pred_proba = model.predict(X_test)

# Get the class with the highest probability
y_pred_class = np.argmax(y_pred_proba, axis=1)

# Classify as "others" if the highest probability is less than 50%
threshold = 0.5
y_pred_class[y_pred_proba.max(axis=1) < threshold] = len(classes)  # "others" class

# Evaluate the new predictions
print(classification_report(y_test_class, y_pred_class))


10/10 ━━━━━━━━━━━━━━━━━━━━ 9s 869ms/step
              precision    recall  f1-score   support

           0       0.93      0.79      0.86        34
           1       0.97      0.82      0.89        44
           2       1.00      1.00      1.00        45
           3       1.00      1.00      1.00        19
           4       0.94      1.00      0.97        17
           5       0.93      1.00      0.96        13
           6       1.00      1.00      1.00        18
           7       0.95      0.82      0.88        22
           8       1.00      0.92      0.96        12
           9       0.95      0.90      0.92        20
          10       0.98      1.00      0.99        50
          11       0.00      0.00      0.00         0

    accuracy                           0.93       294
   macro avg       0.89      0.85      0.87       294
weighted avg       0.97      0.93      0.95       294



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
